In [33]:
# you might have to download tensorflow if you don't have it
#!pip install tensorflow

In [34]:
import tensorflow as tf

In [35]:
daily_sales_number = [21, 22, -108, 31, -2, 32, 34, 31]

In [36]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [37]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-2
32
34
31


In [38]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [39]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [40]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [41]:
tf_dataset = tf_dataset.shuffle(5)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
2232
2232
1584
2448
2304


**We create a batch for our dataset**

In [42]:
for sales_batch in tf_dataset.batch(3):
    print(sales_batch.numpy())

[2232 2448 2304]
[2232 1512 1584]


In [64]:
import tensorflow as tf
daily_sales_numbers = [120, 250, 0, 480, 300, 0, 150]
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x: x > 0) \
                       .map(lambda y: y * 72) \
                       .shuffle(2) \
                       .batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[18000 34560]
[21600  8640]
[10800]


In [46]:
import os
from PIL import Image
import numpy as np

# Create folders
os.makedirs("images/cats", exist_ok=True)
os.makedirs("images/dogs", exist_ok=True)

# Create 2 dummy images
for i in range(2):
    img = Image.fromarray(np.random.randint(0, 255, (64, 64, 3), dtype=np.uint8))
    img.save(f"images/cats/cat{i}.jpg")
    img.save(f"images/dogs/dog{i}.jpg")

# Now this will work
import tensorflow as tf
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

for path in images_ds.as_numpy_iterator():
    print(path)


b'images/cats/cat0.jpg'
b'images/cats/cat1.jpg'
b'images/dogs/dog0.jpg'
b'images/dogs/dog1.jpg'


In [47]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [48]:
image_count = len(images_ds)
image_count

4

In [49]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [50]:
for file in images_ds.take(3):
    print(file.numpy())

b'images/cats/cat0.jpg'
b'images/cats/cat1.jpg'
b'images/dogs/dog0.jpg'


In [51]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images/cats/cat1.jpg'
b'images/dogs/dog0.jpg'
b'images/cats/cat0.jpg'


In [52]:
class_names = ["cat","dog"]

In [53]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [54]:
len(train_ds)

3

In [55]:
len(test_ds)

1

In [56]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [59]:
get_label("images/dog/20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [60]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [68]:
import os
import numpy as np
from PIL import Image

# Create folders
os.makedirs("images/dog", exist_ok=True)
os.makedirs("images/cat", exist_ok=True)

# Create and save dummy images
for i in range(2):
    img = Image.fromarray(np.random.randint(0, 255, (128, 128, 3), dtype=np.uint8))
    img.save(f"images/dog/dog{i}.jpg")
    img.save(f"images/cat/cat{i}.jpg")

In [70]:
def process_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [71]:
def scale(image, label):
    return image/255, label

In [72]:
train_ds = train_ds.map(scale)

In [76]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.5372549 0.7176471 0.5372549]
****Label:  b'cats'
****Image:  [0.80784315 0.8784314  0.4392157 ]
****Label:  b'dogs'
****Image:  [0.5372549 0.7176471 0.5372549]
****Label:  b'dogs'


2025-10-16 17:16:19.808917: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
